In [30]:
# Importação dos datasets e bibliotecas
from sklearn.datasets import load_iris, load_wine, load_breast_cancer, load_digits, load_diabetes, fetch_california_housing, fetch_openml
import pandas as pd

# Função para carregar os datasets
def carregar_dataset(nome_dataset):
    if nome_dataset == 'iris':
        data = load_iris()
        X, y = data.data, data.target
        class_names = data.target_names
    
    elif nome_dataset == 'wine':
        data = load_wine()
        X, y = data.data, data.target
        class_names = data.target_names
    
    elif nome_dataset == 'breast_cancer':
        data = load_breast_cancer()
        X, y = data.data, data.target
        class_names = data.target_names
    
    elif nome_dataset == 'digits':
        data = load_digits()
        X, y = data.data, data.target
        class_names = [str(i) for i in range(10)]
    
    elif nome_dataset == 'diabetes':
        data = load_diabetes()
        X, y = data.data, data.target
        class_names = ['Progression']  # Dados de regressão, podem ser classificados em faixas
    
    elif nome_dataset == 'boston_housing':
        data = fetch_openml(name='boston', version=1)
        X, y = data.data, data.target
        class_names = ['Price']  # Regressão, mas pode ser categorizada
    
    elif nome_dataset == 'california_housing':
        data = fetch_california_housing()
        X, y = data.data, data.target
        class_names = ['Price']
    
    elif nome_dataset == 'wine_quality':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv", sep=';')
        X = data.drop(columns=['quality'])
        y = data['quality']
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'heart_disease':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data", header=None, na_values="?")
        data = data.dropna()  # Remove missing values
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = sorted(y.unique().tolist())
    
    elif nome_dataset == 'banknote_authentication':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt", header=None)
        X = data.iloc[:, :-1]
        y = data.iloc[:, -1]
        class_names = ['Legitimate', 'Forgery']
    
    elif nome_dataset == 'parkinsons':
        data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data")
        X = data.drop(columns=['status', 'name'])
        y = data['status']
        class_names = ['Healthy', 'Parkinsons']
    
    else:
        raise ValueError("Nome do dataset não reconhecido. Escolha um dataset válido.")
    
    return pd.DataFrame(X), y, class_names

# Exemplo: escolha o dataset desejado
nome_dataset = 'breast_cancer'  # Modifique para o dataset desejado
X, y, class_names = carregar_dataset(nome_dataset)
print(f"Dataset {nome_dataset} carregado com sucesso.")
print("Classes:", class_names)
print("Amostras:", X.shape[0], "| Atributos:", X.shape[1])


Dataset breast_cancer carregado com sucesso.
Classes: ['malignant' 'benign']
Amostras: 569 | Atributos: 30


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np

# Função para transformar o dataset em um problema binário (classe 0 contra as outras)
def transformar_problema_binario(y, classe_pos):
    return np.where(y == classe_pos, 1, 0)

# Função para realizar análise de PI-explicação nas instâncias de teste
def analisar_instancias(X, y, class_names, classe_pos=0, instancia_para_analisar=None):
    
    # Transforma o problema em binário: classe `classe_pos` vs. as outras
    y_binario = transformar_problema_binario(y, classe_pos)

    # Divide o dataset em treino e teste (80% treino, 20% teste)
    X_train, X_test, y_train, y_test = train_test_split(X, y_binario, test_size=0.2, random_state=42)
    
    # Treina o modelo no conjunto de treino para um problema binário
    modelo = LogisticRegression(max_iter=200)
    modelo.fit(X_train, y_train)
    # Seleciona as instâncias para análise (conjunto de teste)
    num_instancias = len(X_test)
    instancias_para_analisar = range(num_instancias) if instancia_para_analisar is None else [instancia_para_analisar]
    
    # Loop para analisar cada instância selecionada
    for idx in instancias_para_analisar:
        Vs = X_test.iloc[idx].to_dict()  # Valores da instância específica
        instancia_test = X_test.iloc[[idx]]  # Mantém o formato DataFrame

        # Calcula `gamma_A` usando `decision_function` para a classe positiva
        gamma_A = modelo.decision_function(instancia_test)[0]
        
        # Cálculo do valor delta para cada feature
        delta = []
        w = modelo.coef_[0]  # Pesos do modelo treinado
        for i, feature in enumerate(X.columns):
            if w[i] < 0:
                delta.append((Vs[feature] - X[feature].max()) * w[i])
            else:
                delta.append((Vs[feature] - X[feature].min()) * w[i])

        # Calcula R como a soma dos deltas menos gamma_A
        R = sum(delta) - gamma_A
        
        # Computa a PI-explicação para a instância atual
        Xpl = one_explanation(Vs, delta, R, X_test.columns)
        
        # Imprime os resultados
        classe_verdadeira = y_test[idx]  # Usando indexação direta para o valor da classe
        print(f"\nInstância {idx}:")
        print(f"Classe verdadeira (binária): {classe_verdadeira}")
        print(f"PI-Explicação: ")
        for item in Xpl:
            print(f"---- {item}")
        if not Xpl:
            print('_No-PI-explanation_'*3)
        

# Função para calcular a PI-explicação
def one_explanation(Vs, delta, R, feature_names):
    Xpl = []  # Inicializa a lista de PI-explicação
    delta_sorted = sorted(enumerate(delta), key=lambda x: abs(x[1]), reverse=True)
    R_atual = R  # Inicializa o limiar atual
    Idx = 0  # Inicializa o índice para iterar
    
    while R_atual >= 0 and Idx < len(delta_sorted):
        sorted_idx, delta_value = delta_sorted[Idx]  # Índice e valor do delta
        feature = feature_names[sorted_idx]  # Nome da característica correspondente
        feature_value = Vs[feature]  # Valor da característica na instância Vs
        Xpl.append(f"{feature} - {feature_value}")
        R_atual -= delta_value  # Atualiza o limiar atual
        Idx += 1
    return Xpl

# Executar a análise de PI-explicação no conjunto de teste do dataset selecionado
# Certifique-se de ter carregado o dataset na célula 'dataset' antes de executar esta célula
analisar_instancias(X, y, class_names, classe_pos=0)
# Gera uma lista com os nomes das características influentes na PI-explicação





Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
---- 20 - 14.97
---- 0 - 12.47
---- 23 - 677.9
---- 21 - 24.64
---- 2 - 81.09
---- 11 - 1.044
---- 3 - 481.9
---- 1 - 18.6
---- 22 - 96.05
---- 13 - 30.29
---- 26 - 0.2671
---- 12 - 2.497
---- 25 - 0.2378
---- 28 - 0.3014
---- 27 - 0.1015
---- 6 - 0.08005
---- 5 - 0.1058
---- 24 - 0.1426
---- 8 - 0.1925
---- 7 - 0.03821
---- 4 - 0.09965
---- 29 - 0.0875
---- 16 - 0.02701
---- 10 - 0.3961
---- 15 - 0.01911
---- 9 - 0.06373
---- 18 - 0.01782
---- 17 - 0.01037
---- 14 - 0.006953
---- 19 - 0.003586

Instância 1:
Classe verdadeira (binária): 1
PI-Explicação: 
---- 23 - 1866.0
---- 0 - 18.94
---- 20 - 24.86
---- 2 - 123.6
---- 13 - 96.05
---- 22 - 165.9

Instância 2:
Classe verdadeira (binária): 1
PI-Explicação: 
---- 23 - 1156.0
---- 20 - 19.26
---- 0 - 15.46
---- 2 - 101.7
---- 21 - 26.0
---- 11 - 0.7859
---- 22 - 124.9
---- 3 - 748.9
---- 13 - 48.31

Instância 3:
Classe verdadeira (binária): 0
PI-Explicação: 
---- 20 - 12.88
---

c:\Users\gleilsonpedro\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [28]:
from copy import deepcopy

def testar_perturbacao(instancia, pi_explicacao, modelo, delta=0.1):
    """
    Função para testar a robustez das PI-explicações por meio de perturbação
    nas características influentes.

    Parâmetros:
    - instancia: dicionário com os valores das características da instância a ser explicada (Vs)
    - pi_explicacao: lista de características importantes da PI-explicação
    - modelo: modelo treinado (LogisticRegression ou outro classificador linear)
    - delta: valor da perturbação a ser aplicado em cada característica (padrão 0.1)
    """
    print("\nTeste de Perturbação para Robustez da PI-Explicação:\n")
    
    # Copiar a instância original para preservar o estado original
    perturbed_instance = deepcopy(instancia)
    
    # Itera sobre as características mais influentes da PI-explicação
    for feature in pi_explicacao:
        # Aplica perturbação à característica relevante
        perturbed_instance[feature] += delta
        
        # Converte a instância perturbada para DataFrame para compatibilidade com o modelo
        perturbed_df = pd.DataFrame([perturbed_instance])
        
        # Realiza a previsão para a instância perturbada
        pred_perturbada = modelo.predict(perturbed_df)[0]
        
        # Realiza a previsão para a instância original
        pred_original = modelo.predict(pd.DataFrame([instancia]))[0]
        
        # Resultado da perturbação
        print(f"Perturbação na característica '{feature}':")
        print(f"- Previsão original: {pred_original}")
        print(f"- Previsão após perturbação: {pred_perturbada}")
        
        # Verifica se a perturbação alterou a previsão
        if pred_perturbada != pred_original:
            print(f" -> A perturbação de '{feature}' alterou a previsão para {pred_perturbada}\n")
        else:
            print(f" -> A perturbação de '{feature}' NÃO alterou a previsão\n")
        
        # Reverte a perturbação para testar a próxima característica
        perturbed_instance[feature] -= delta

# Exemplo de chamada para testar a robustez:
# Certifique-se de passar as características da PI-explicação sem os valores (somente nomes)
# e de que `Vs`, `Xpl`, e `modelo` estejam disponíveis no notebook

# Chame a função e armazene as variáveis retornadas
Vs, caracteristicas_pi_explicacao, modelo = analisar_instancias(X, y, class_names, classe_pos=0)

# Agora chame o teste de perturbação
testar_perturbacao(Vs, caracteristicas_pi_explicacao, modelo)




Instância 0:
Classe verdadeira (binária): 0
PI-Explicação: 
---- 2 - 4.7
---- 3 - 1.2
---- 0 - 6.1
---- 1 - 2.8

Teste de Perturbação para Robustez da PI-Explicação:



KeyError: '2'